In [8]:
from tools import overview
import importlib
import rawFileReader
from tools import Reconstruction_tools as RTools
import pickle
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path


In [9]:
#Save chunks
files = ["proAnubis_240815_1759.raw", "proAnubis_240818_1125.raw","proAnubis_240829_2026.raw", "proAnubis_240818_1325.raw", "proAnubis_240829_2026.raw", "proAnubis_240829_2026.raw"]
rpc = 0

storage_name = f"data/chunks_hv{rpc}.pkl"
file_name = files[rpc]

hv_file = 'data/hvScan.csv'  # Replace with your file path
hv_data = pd.read_csv(hv_file, usecols=["start_"+str(rpc),"end_"+str(rpc),"voltage_"+str(rpc)])

#rpc 0: 2024-08-15 17:24:22	
#rpc 1	2024-08-18 11:39:42
#rpc 2	2024-08-18 12:22:21 


In [ ]:
fReader = rawFileReader.fileReader(dir_path+"data/"+file_name) # load in the classs object
total_chunks = []
for start, end, voltage in hv_data.values:
    if type(start) == float:
        break
    print("Voltage:", voltage)
    start = "2024-08-15 " + start 
    end = "2024-08-15 "+ end
    chunks, times, fReader = overview.get_chunks(file_name, fReader=fReader, start=start, end=end)
    print(len(chunks[0]))
    total_chunks.append((voltage, times, chunks))

: 

In [ ]:
with open(storage_name, "wb") as outp:
    pickle.dump(total_chunks, outp)
print("Chunks Saved")

Chunks Saved


In [11]:
with open(storage_name, "rb") as inp:
    total_chunks = pickle.load(inp)
print("Chunks Loaded")

Chunks Loaded


In [7]:
with open(dir_path + "//data//tot_mean.pkl", "rb") as f:
    tot_mean = pickle.load(f)
with open(dir_path + "//data//tot_std.pkl", "rb") as f:
    tot_std = pickle.load(f)

In [15]:
importlib.reload(overview)

complete_data = []

with PdfPages(f"{rpc}.pdf") as pdf: #trash pdf, so I do not need to click on
    for voltage, times, chunks in total_chunks:
        results_dict = {}
        print(f"# Chunks for {round(voltage)}:", len(chunks))
        if len(chunks) == 1:
            print("Only one chunk, skipping")
            continue

        #time_hits, bvg = overview.tdc_times_stats(chunks)
        
        time_hist_phi = []
        time_hist_eta = []
        time_hist_min = []

        reconstructor = RTools.Reconstructor(chunks[0], 0)
        reconstructor.std_tot = tot_std
        reconstructor.mean_tot = tot_mean
        for chunk_num, chunk in enumerate(chunks[1:]):
            reconstructor.update_event(chunk, chunk_num)
            clusters = reconstructor.cluster()
            for evt_num, evt_cluster in enumerate(clusters):
                for rpc in range(6):
                    for cluster in evt_cluster[rpc]:
                        if cluster.size <= 3:
                            time_hist_min.append(cluster.time[0])
                            time_hist_phi.extend([hit.time for hit in cluster.hits[0]])
                            time_hist_eta.extend([hit.time for hit in cluster.hits[1]])
                
        hist, bins = np.histogram(time_hist_min, bins=100, range=(150, 350))
        results_dict["time_hist_min"] = (hist, bins)
        hist, bins = np.histogram(time_hist_phi, bins=100, range=(150, 350))
        results_dict["time_hist_phi"] = (hist, bins)
        hist, bins = np.histogram(time_hist_eta, bins=100, range=(150, 350))
        results_dict["time_hist_eta"] = (hist, bins)

        """
        cluster_size, error, hist = overview.cluster_size(chunks, residual=False, seperate = True)
        results_dict["cluster_size"] = (cluster_size, error, hist)
        """
        
        #good, bad = overview.abs_bvg_hits(chunks[:100], times[:100], per_rpc=True, pdf = pdf)[rpc]
        
        
        #hist, std = overview.hit_time_hist(chunks, per_rpc= True, pdf=pdf)
        #results_dict["hit_time_hist"] = (hist, std)
        results_dict["voltage"] = voltage
        if results_dict:
            complete_data.append(results_dict)


        

# Chunks for 3000: 1
Only one chunk, skipping
# Chunks for 4000: 1
Only one chunk, skipping
# Chunks for 4500: 1
Only one chunk, skipping
# Chunks for 4750: 16
# Chunks for 5000: 304
# Chunks for 5100: 619
# Chunks for 5200: 940
# Chunks for 5250: 2186
# Chunks for 5300: 720
# Chunks for 5350: 874
# Chunks for 5400: 1049
# Chunks for 5450: 1468
# Chunks for 5500: 980
# Chunks for 5550: 1304


C:\Users\jony\AppData\Local\Temp\ipykernel_29932\2666536752.py:5: MatplotlibDeprecationWarning: Keeping empty pdf files is deprecated since 3.8 and support will be removed in 3.10.
  with PdfPages(f"{rpc}.pdf") as pdf: #trash pdf, so I do not need to click on


KeyboardInterrupt: 

In [12]:
import matplotlib.pyplot as plt
import numpy as np
selected_voltages = [5600, 5800, 6000]

for channel_set in range(2): #0 - phi, 1 - eta
    for volt_data in complete_data:
        if volt_data["voltage"] in selected_voltages and "cluster_size" in volt_data.keys():
            hist, bins = np.histogram(volt_data["cluster_size"][2][channel_set][rpc], bins=range(1, max(volt_data["cluster_size"][2][channel_set][rpc])+2))
            plt.step(bins[:-1], hist, label="Voltage: "+str(volt_data["voltage"])+"V", alpha=0.5)
    
    plt.legend()
    plt.xlabel("Cluster Size")
    plt.ylabel("Counts")
    plt.yscale("log")
    plt.title(f"{'Eta' if channel_set else 'Phi'} Cluster Size Distribution RPC {rpc}")
    plt.show()




In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))  # Create a new figure

# Plot In-peak and Off-peak hits in the first figure

good = []
bad = []
voltage = []
for volt_data in complete_data:
    if "counts" not in volt_data.keys():
        continue
    good.append(sum(volt_data["counts"][0])/100)
    bad.append(sum(volt_data["counts"][1])/100)
    voltage.append(volt_data["voltage"])

plt.plot(voltage, good, label="In-peak hits", color="blue")
plt.plot(voltage, bad, label="Off-peak hits", color="red")
plt.title(f"Off-peak and In-peak hits (RPC {rpc})")
plt.xlabel("Voltage (V)")
plt.ylabel("Counts per chunk")
plt.legend()
plt.show()

print("Good",good )
print("Bad", bad)
print("Voltage", voltage)

In [16]:
# Plot Efficiency in the third figure
plt.figure(figsize=(10, 6))  # Create a new figure
efficiencies = []
voltage = []
for volt_data in complete_data:
    if "efficiency" not in volt_data.keys():
        continue
    efficiencies.append(volt_data["efficiency"][rpc])
    voltage.append(volt_data["voltage"])
plt.plot(voltage, [eff[rpc] for eff in efficiencies], label="Efficiency", color="red")
plt.title(f"Efficiency (RPC {rpc})")
plt.xlabel("Voltage (V)")
plt.ylabel("Efficiency")
plt.legend()
plt.show()

print("Efficiencies:", efficiencies)
print("Voltages:", voltage)

Efficiencies: [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
Voltages: [4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]


In [17]:
# Plot Cluster size in the second figure
clusters = []
errors = []
voltage = []
for volt_data in complete_data:
    if "cluster_size" not in volt_data.keys():
        continue
    clusters.append(volt_data["cluster_size"][0][rpc])
    errors.append(volt_data["cluster_size"][1][rpc])
    voltage.append(volt_data["voltage"])

plt.figure(figsize=(10, 6))  # Create a new figure
plt.plot(voltage, clusters, label="Cluster size", color="red")
plt.errorbar(voltage, clusters, yerr=errors, fmt='o', color="red")
plt.title(f"Cluster size (RPC {rpc})")
plt.xlabel("Voltage (V)")
plt.ylabel("Cluster size")
plt.legend()
plt.show()

print("Cluster size:", clusters)
print("Errors:", errors)
print("Voltages", voltage)


Cluster size: [np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan)]
Errors: [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
Voltages [4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]


In [35]:
#Plot Histograms
rpc = 4
selected_voltages = [5600, 5800, 6000]
plt.figure(figsize=(10, 6))  # Create a new figure
for i, volt_data in enumerate(complete_data):
    if volt_data["voltage"] in selected_voltages and "hit_time_hist" in volt_data.keys():
        hist = volt_data["hit_time_hist"][0][0]
        plt.plot(hist[rpc][0][:-1], hist[rpc][1], label=f"Voltage {volt_data['voltage']} V")
        plt.title(f"Hit time histogram (RPC {rpc})")
        plt.xlabel("Time (25/32 ns)")
        plt.yscale("log")
        plt.ylabel("Counts (normalised)")
plt.legend()
plt.show()

In [33]:
stds = []
voltages = []
for volt_data in complete_data:
    if "hit_time_hist" in volt_data.keys():
        stds.append(volt_data["hit_time_hist"][1][0][rpc])
        voltages.append(volt_data["voltage"])

plt.plot(voltages, stds , label="Standard deviation")
plt.xlabel("Voltage (V)")
plt.ylabel("Standard deviation")
plt.title(f"Spread of the hit time peak (RPC {rpc})")
plt.legend()
plt.show()

print("Stds:", stds)
print("Voltages:", voltages)

Stds: [np.float64(198.28548465012042), np.float64(171.2463156034313), np.float64(195.05149006410264), np.float64(187.84188200615515), np.float64(200.2884212022692), np.float64(214.99125021298082), np.float64(223.63960361884546), np.float64(245.56708406769977), np.float64(253.31949145527304), np.float64(254.8483116389822), np.float64(256.4219398001788), np.float64(276.913536169867), np.float64(268.5446704210353), np.float64(271.66737938804954), np.float64(276.4722254322756), np.float64(278.5488587842489), np.float64(282.0593934539096), np.float64(272.9047684572004), np.float64(270.0153303980897), np.float64(261.36739393233586)]
Voltages: [4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]


In [131]:
#Save chunks
files = ["proAnubis_240815_1759.raw", "proAnubis_240818_1125.raw", "proAnubis_240818_1325.raw"]
rpc = 2

storage_name = f"chunks_hv{rpc}.pkl"
file_name = files[rpc]

hv_file = 'data/hvScan.csv'  # Replace with your file path
hv_data = pd.read_csv(hv_file)
#hv_data = pd.read_csv(hv_file, usecols=["start_"+str(rpc),"end_"+str(rpc),"voltage_"+str(rpc)])

#rpc 0: 2024-08-15 17:24:22	
#rpc 1	2024-08-18 11:39:42
#rpc 2	2024-08-18 12:22:21 


import numpy as np
import matplotlib.pyplot as plt
cl0 = [np.float64(1.0339622641509434), np.float64(1.045432730763078), np.float64(1.0991290983606556), np.float64(1.1098737353933026), np.float64(1.158373989173294), np.float64(1.199706026457619), np.float64(1.243627828510654), np.float64(1.3051052260598126), np.float64(1.3432812085873311), np.float64(1.4434467018370936), np.float64(1.5500040730985418), np.float64(1.6012395661518468), np.float64(1.7499497487437186), np.float64(1.9432747311988978), np.float64(2.1125484678996673), np.float64(2.418690392760291), np.float64(2.816230601735043), np.float64(3.3619210977701544), np.float64(4.264796643667002), np.float64(5.270159270760293)]
err0 = [np.float64(0.011126854946426285), np.float64(0.00797484375042157), np.float64(0.010639097472394752), np.float64(0.006399245040924702), np.float64(0.005373077722241347), np.float64(0.011612885000507504), np.float64(0.011525592304870512), np.float64(0.012537685498550555), np.float64(0.011259031916725452), np.float64(0.01603256469828638), np.float64(0.016418504051676275), np.float64(0.016081504890198023), np.float64(0.019236149879349896), np.float64(0.02214773563960034), np.float64(0.026127083422149457), np.float64(0.028637597625313208), np.float64(0.03627862360495955), np.float64(0.03986771546091226), np.float64(0.05496790069215523), np.float64(0.06452445127451842)]
eff_0 = [0.7142857142857143, 0.631578947368421, 0.7407407407407407, 0.800498753117207, 0.837734404898584, 0.8513661202185793, 0.8619281045751634, 0.8058705803869246, 0.7712418300653595, 0.7727272727272727, 0.8007478632478633, 0.8371369294605809, 0.06060606060606061, 0.8083756345177665, 0.7622107969151671, 0.6454965357967667, 0.6234747239976758, 0.5983076157292185, 0.5689544579858884, 0.6159267089499648]
good_0 = [27.23, 176.75, 187.79, 208.36, 217.25, 219.96, 231.34, 237.86, 244.11, 260.15, 282.66, 307.92, 334.27, 358.79, 387.47, 420.82, 458.28, 558.12, 707.28, 952.21]
bad_0 = [0.36, 7.19, 10.49, 13.81, 15.59, 17.85, 20.35, 25.15, 26.74, 27.65, 39.5, 38.18, 45.49, 48.32, 61.55, 60.18, 68.13, 77.08, 101.84, 110.29]
std_0 = [np.float64(116.44640183214655), np.float64(147.14398853196343), np.float64(130.4686001108644), np.float64(60.85581179078705), np.float64(147.10844128432203), np.float64(175.56184808696304), np.float64(195.82146789505632), np.float64(203.24078063739753), np.float64(222.7283138879541), np.float64(216.74833196589313), np.float64(233.39542971654942), np.float64(238.1559128032108), np.float64(248.41570803066662), np.float64(254.446597678295), np.float64(254.71945322400353), np.float64(264.04218057542863), np.float64(254.63390691215753), np.float64(260.4522079461875), np.float64(261.20382309648596), np.float64(268.37842991368245), np.float64(265.0101410268362), np.float64(256.88603378265293), np.float64(246.9555944252669)]
voltages_0 = [3000.0, 4000.0, 4500.0, 4750.0, 5000.0, 5100.0, 5200.0, 5250.0, 5300.0, 5350.0, 5400.0, 5450.0, 5500.0, 5550.0, 5600.0, 5650.0, 5700.0, 5750.0, 5800.0, 5850.0, 5900.0, 5950.0, 6000.0]


cl1 = [np.float64(1.2212389380530972), np.float64(1.1514522821576763), np.float64(1.047808764940239), np.float64(1.074378831881252), np.float64(1.0793083854208945), np.float64(1.130030772943945), np.float64(1.1806598517513687), np.float64(1.2741918045194076), np.float64(1.3286349856293556), np.float64(1.3650258038904328), np.float64(1.4437348498974454), np.float64(1.5276988398916787), np.float64(1.6550848547184367), np.float64(1.7486767031002957), np.float64(1.9628453593196833), np.float64(2.2378623115171603), np.float64(2.5977705099267583), np.float64(3.288475477168398), np.float64(4.1703055084091964), np.float64(5.144818364163138)]
err1 = [np.float64(0.15365365868781636), np.float64(0.050306133269046746), np.float64(0.01085851940062489), np.float64(0.012610236466712046), np.float64(0.007588714747234421), np.float64(0.009364968167379703), np.float64(0.010283760210871348), np.float64(0.012014437176205687), np.float64(0.013788835005604861), np.float64(0.012602875350185448), np.float64(0.017094625432900766), np.float64(0.016288766736714727), np.float64(0.01812416969437741), np.float64(0.019781668576846993), np.float64(0.023790335485086862), np.float64(0.029631820011282582), np.float64(0.03495556082164944), np.float64(0.044105284020961893), np.float64(0.05386122345286682), np.float64(0.04904885980725799)]
eff_1 = [0, 0.6363636363636364, 0.6, 0.6492890995260664, 0.6837416481069042, 0.7642526964560863, 0.8214285714285714, 0.7619439868204283, 0.7778810408921933, 0.8096013018714402, 0.8154020385050963, 0.8282910874897792, 0.7958030669895076, 0.7861271676300579, 0.7273449920508744, 0.6556451612903226, 0.6175889328063241, 0.6141078838174274, 0.6260543580131209, 0.6921641791044776]
good_1 = [23.11, 101.19, 143.96, 161.91, 178.94, 195.78, 209.24, 210.39, 224.92, 246.56, 265.62, 278.61, 294.74, 311.62, 325.12, 350.22, 413.4, 535.85, 680.4, 946.93]
bad_1 = [0.82, 2.85, 4.97, 10.8, 11.69, 13.95, 19.5, 22.54, 28.8, 30.71, 39.49, 39.49, 43.2, 48.55, 57.97, 48.03, 68.53, 78.68, 94.05, 143.15]
std_1 = [np.float64(33.62848900345012), np.float64(75.40251771826425), np.float64(20.23976986093638), np.float64(98.9403675674452), np.float64(157.65312727971587), np.float64(25.425914261172096), np.float64(155.12154453706253), np.float64(122.58264578206251), np.float64(143.98342398843823), np.float64(176.8360062555137), np.float64(190.7766510634326), np.float64(208.47690585506578), np.float64(231.04675647537343), np.float64(239.5889561940645), np.float64(244.3349516634308), np.float64(242.81730853509387), np.float64(255.74022309532998), np.float64(259.7230028227785), np.float64(263.26304771075047), np.float64(263.73677604056456), np.float64(272.60875767022014), np.float64(264.79901299335796), np.float64(270.115786313314), np.float64(267.52328190045745), np.float64(259.5640546638369), np.float64(256.75948891329324)]
voltages_1 = [4000, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]

cl2 = [np.float64(1.2426805465191932), np.float64(1.1026037638566641), np.float64(1.0717778305245669), np.float64(1.078498755276545), np.float64(1.1012601260126014), np.float64(1.150940736322724), np.float64(1.2325704415416172), np.float64(1.2810768426555148), np.float64(1.3504882892942458), np.float64(1.4278187870893038), np.float64(1.5336447215991718), np.float64(1.6422477069283326), np.float64(1.8094636989734356), np.float64(1.9863480560180198), np.float64(2.1780307743353657)]
err2 = [np.float64(0.05284236695895426), np.float64(0.013833909627537408), np.float64(0.005910129368264823), np.float64(0.004334482293023012), np.float64(0.003404930544784941), np.float64(0.003349760146393746), np.float64(0.004875910061957942), np.float64(0.0051769679836586755), np.float64(0.0057941181596519245), np.float64(0.0066690683402625215), np.float64(0.008848887987821364), np.float64(0.009000379354788007), np.float64(0.009870044862030278), np.float64(0.00940555685055206), np.float64(0.013261509746135937)]
eff_2 = [0.2823529411764706, 0.2684729064039409, 0.3488372093023256, 0.39231824417009603, 0.5069294733600246, 0.679410593389088, 0.7110944246292472, 0.6984031936127745, 0.6787052810902896, 0.6139962540439299, 0.5336206896551724, 0.5263077393698379, 0.5024705435195743, 0.5202688510817055, 0.5354093420391763]
good_2 = [21.23, 88.8, 128.09, 127.53, 150.66, 187.99, 212.2, 221.82, 221.84, 228.64, 225.27, 240.31, 257.37, 285.93, 327.48]
bad_2 = [1.29, 6.13, 9.92, 13.2, 15.92, 21.61, 28.75, 36.93, 34.19, 39.12, 35.15, 37.76, 49.98, 57.14, 56.99]
std_2 = [np.float64(149.370138953448), np.float64(193.14370170119776), np.float64(201.72769030199757), np.float64(224.35787744702475), np.float64(237.48455276479595), np.float64(246.33221676807943), np.float64(261.32548875781913), np.float64(280.7314198206911), np.float64(274.9229756104916), np.float64(281.08858084233566), np.float64(286.38562397612975), np.float64(285.0251237585091), np.float64(292.19252896413644), np.float64(294.28433544948666), np.float64(290.1213073476663)]
voltages_2 = [4800.0, 4900.0, 5000.0, 5100.0, 5200.0, 5300.0, 5400.0, 5450.0, 5500.0, 5550.0, 5600.0, 5650.0, 5700.0, 5750.0, 5800.0]

cl3 = [np.float64(1.3109243697478992), np.float64(1.4304932735426008), np.float64(1.088797108931337), np.float64(1.0732620320855615), np.float64(1.0889143293264645), np.float64(1.140652965500216), np.float64(1.2034606205250598), np.float64(1.300078919270229), np.float64(1.3564630027632791), np.float64(1.4238995328104238), np.float64(1.585788485880145), np.float64(1.614696409399976), np.float64(1.7720491102649778), np.float64(1.9402696793002916), np.float64(2.200837122032186), np.float64(2.4071046264388887), np.float64(2.8758147771372595), np.float64(3.5219217537402994), np.float64(4.607378273061656), np.float64(5.997637872773203)]
err3 = [np.float64(0.13743948558122332), np.float64(0.17042343068733465), np.float64(0.029088649923025792), np.float64(0.014510821401138367), np.float64(0.01037693249897322), np.float64(0.00932813750248753), np.float64(0.01002816662956866), np.float64(0.012962796812449724), np.float64(0.015681357316640058), np.float64(0.015622954630547714), np.float64(0.021914920916389528), np.float64(0.019423017007949815), np.float64(0.02219151199022226), np.float64(0.024554762448446458), np.float64(0.028902655790087), np.float64(0.031205380199562274), np.float64(0.03841785594435168), np.float64(0.047352697168423274), np.float64(0.057293592625362544), np.float64(0.06856915514193454)]
eff_3 = [0.3333333333333333, 0, 0.6046511627906976, 0.6592592592592592, 0.7282229965156795, 0.8059701492537313, 0.8470209339774557, 0.8125894134477826, 0.8591352859135286, 0.7796610169491526, 0.7790262172284644, 0.83729662077597, 0.8648960739030023, 0.8652173913043478, 0.8177676537585421, 0.7995712754555199, 0.7423913043478261, 0.6504524886877828, 0.6239035087719298, 0.6424180327868853]
good_3 = [39.41, 118.95, 167.0, 178.56, 195.79, 211.84, 229.26, 247.03, 251.18, 267.39, 281.43, 314.74, 349.06, 390.47, 420.62, 477.23, 521.25, 645.4, 786.85, 1058.32]
bad_3 =[1.24, 3.28, 5.57, 9.0, 12.72, 14.29, 17.94, 22.83, 22.58, 30.95, 27.29, 30.73, 42.35, 41.41, 49.63, 57.62, 68.54, 78.72, 95.06, 103.75]
std_3 = [np.float64(22.580777820499762), np.float64(36.26761181903454), np.float64(115.76890863771862), np.float64(45.15559029300194), np.float64(15.544845883103225), np.float64(50.23009591777843), np.float64(109.6711771245793), np.float64(117.64960279069595), np.float64(139.9139886519241), np.float64(164.5487685457531), np.float64(179.88285706818914), np.float64(197.46726684488053), np.float64(204.03595753014383), np.float64(213.83843936997752), np.float64(222.52963942577753), np.float64(228.13897591693993), np.float64(236.74615068110663), np.float64(236.3079104812058), np.float64(241.65472667069506), np.float64(252.93947698782003), np.float64(242.48333735286218), np.float64(245.4035123716854), np.float64(241.87816539584296), np.float64(240.27277861641534), np.float64(230.55177350620454), np.float64(225.56577025801803)]
voltages_3 = [4000, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]


cl4 = [np.float64(1.229393468118196), np.float64(1.0739962940086474), np.float64(1.077679826242494), np.float64(1.0722836793453554), np.float64(1.1058748719398028), np.float64(1.164411914689384), np.float64(1.2074569888579676), np.float64(1.2731271833354898), np.float64(1.3228828298258044), np.float64(1.3880580391045048), np.float64(1.4323709832732978), np.float64(1.5331179181537822), np.float64(1.630655118035359), np.float64(1.747050235741809), np.float64(1.9155770133440622), np.float64(2.082685400797253), np.float64(2.3691230097686296), np.float64(2.7308624049520462), np.float64(3.3108430134837334)]
err4 = [np.float64(0.05526648654725857), np.float64(0.008409939018719743), np.float64(0.009305563276871634), np.float64(0.004460948803990459), np.float64(0.0041631648585835545), np.float64(0.00522557924381099), np.float64(0.004368185596520588), np.float64(0.004823697859814168), np.float64(0.006195982484626984), np.float64(0.006948988581460802), np.float64(0.006896873251404145), np.float64(0.008051279912303847), np.float64(0.008652393520111944), np.float64(0.008072802275942691), np.float64(0.010772254242382847), np.float64(0.011197554996159008), np.float64(0.013074844647443604), np.float64(0.015156791913584827), np.float64(0.01823735545006964)]
eff_4 = [0.17391304347826086, 0.13861386138613863, 0.32432432432432434, 0.3763586956521739, 0.5771947527749748, 0.5413879598662207, 0.8268062120189061, 0.8748775710088149, 0.8614949748743719, 0.7784568372803667, 0.8207518796992481, 0.8721301020408163, 0.9262478968031408, 0.9230608404766882, 0.9071899169292467, 0.9002849002849003, 0.8785134291068083, 0.8619700350653491, 0.8699551569506726]
good_4 =  [18.15, 93.93, 105.0, 111.33, 151.9, 141.64, 219.06, 248.67, 251.66, 240.63, 261.64, 291.05, 331.42, 355.48, 381.48, 402.98, 462.2, 552.28, 673.67]
bad_4 = [1.57, 7.92, 8.77, 10.64, 14.94, 15.69, 26.84, 39.07, 37.36, 36.3, 42.45, 42.79, 56.95, 67.12, 63.67, 78.66, 82.45, 89.34, 104.26]
std_4 = [np.float64(185.5278210278003), np.float64(207.41997554442491), np.float64(203.59029805983602), np.float64(210.30613082125964), np.float64(224.56450001930966), np.float64(230.0687972155798), np.float64(253.31769684365872), np.float64(259.206997164944), np.float64(261.3947709063714), np.float64(263.5191734360712), np.float64(282.08001676606654), np.float64(273.9868239150408), np.float64(277.5160364731628), np.float64(279.3208272774941), np.float64(278.68928971167094), np.float64(278.97531136854906), np.float64(276.0689274809762), np.float64(276.89059440482686), np.float64(266.8915187688332)]
voltages_4 = [4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]

cl5 = [np.float64(1.172544080604534), np.float64(1.0921935711762627), np.float64(1.0795882669537136), np.float64(1.0735568993952722), np.float64(1.0929640053325433), np.float64(1.1463041350117886), np.float64(1.2308597643724404), np.float64(1.3322483258997901), np.float64(1.4226306901704469), np.float64(1.5174786248664054), np.float64(1.6257088418091912), np.float64(1.7910224749413741), np.float64(1.987141311877879), np.float64(2.223580702112583), np.float64(2.4503379285389113), np.float64(2.7105627329568986), np.float64(2.9942503846898063), np.float64(3.4240823424354936), np.float64(4.035348844922801), np.float64(4.954357544584916)]
err5 = [np.float64(0.03692048813009653), np.float64(0.013455024100491745), np.float64(0.008847031098445866), np.float64(0.004489594488290236), np.float64(0.003203862534572579), np.float64(0.0038739669204423343), np.float64(0.00539528957870886), np.float64(0.006281869298693577), np.float64(0.007562647889993112), np.float64(0.008258621944831424), np.float64(0.009083147640689819), np.float64(0.010092415890611463), np.float64(0.012404006360859579), np.float64(0.013864660096798412), np.float64(0.015019594355821807), np.float64(0.015334966929104665), np.float64(0.0161493354029492), np.float64(0.018885981889940544), np.float64(0.021222736078301482), np.float64(0.02193409700704491)]
eff_5 =  [0.3333333333333333, 0.47761194029850745, 0.5159474671669794, 0.571858087793145, 0.7550879613659883, 0.6555158324821246, 0.5657689305230289, 0.5373893388585421, 0.564922480620155, 0.5436685122343276, 0.523716814159292, 0.503137747097584, 0.48551800965466024, 0.4774597495527728, 0.4939454183986987, 0.5449600543940166, 0.6106550510973362, 0.6477869467366841, 0.6997716894977168, 0.7024581511084301]
good_5 = [11.45, 33.63, 160.87, 156.79, 196.11, 191.4, 188.62, 185.79, 228.21, 249.39, 239.15, 267.09, 279.93, 315.31, 343.37, 406.8, 513.26, 622.25, 809.67, 1020.27]
bad_5 = [1.54, 4.74, 19.75, 15.2, 20.94, 25.93, 23.68, 30.14, 32.65, 38.14, 39.06, 42.57, 50.57, 47.42, 56.49, 61.35, 83.31, 105.01, 122.97, 156.79]
std_5 = [np.float64(222.07351662810785), np.float64(244.52627256724753), np.float64(242.94639647401755), np.float64(227.89937387919977), np.float64(239.14897222311114), np.float64(258.17844336005936), np.float64(265.35126964193324), np.float64(265.58088472953415), np.float64(261.05820862730224), np.float64(268.8675399615145), np.float64(276.79643883095594), np.float64(279.6126188734449), np.float64(276.27723440466764), np.float64(268.4812732201545), np.float64(273.4995638402457), np.float64(277.97849571734633), np.float64(274.37457537043406), np.float64(267.98310403017064), np.float64(256.9631437831994), np.float64(253.16329570405418)]
voltages_5 = [4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000]


cluster = [cl0, cl1, cl2, cl3, cl4, cl5]
error = [err0, err1, err2, err3, err4, err5]
eff = [eff_0, eff_1, eff_2, eff_3, eff_4, eff_5]
good = [good_0, good_1, good_2, good_3, good_4, good_5]
bad = [bad_0, bad_1, bad_2, bad_3, bad_4, bad_5]
stds = [std_0, std_1, std_2, std_3, std_4, std_5]
voltages = [voltages_0, voltages_1, voltages_2, voltages_3, voltages_4, voltages_5]

#plot cluster size
number = 6
colour = ["red", "blue", "green", "orange", "brown", "orchid"]
labels = ["bottom triplet", "middle triplet", "top triple", "singlet", "bottom doublet",  "top doublet"]
plt.figure(figsize=(10, 6))  # Create a new figure
for rpc in range(number):
    plt.plot(voltages[rpc][len(voltages[rpc])-len(good[rpc]):], good[rpc], label=labels[rpc]+" in-peak", marker = "o", color=colour[rpc])
    plt.plot(voltages[rpc][len(voltages[rpc])-len(bad[rpc]):], bad[rpc], label=labels[rpc]+" off-peak", marker = "x", linestyle = "--", color=colour[rpc])

plt.title("Hit Counts vs. Voltage")
plt.ylabel("hit Count per Chunk")
plt.xlabel("Voltage / V")
plt.xlim(5400, 6000)
plt.ylim(0, 1200)
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))  # Create a new figure
for rpc in range(number):
    plt.plot(voltages[rpc][len(voltages[rpc])-len(cluster[rpc]):], cluster[rpc], label=labels[rpc], marker = "o", color=colour[rpc])
    plt.errorbar(voltages[rpc][len(voltages[rpc])-len(cluster[rpc]):], cluster[rpc], yerr=error[rpc],  marker = "o", color=colour[rpc])
plt.title("Cluster Size vs. Voltage")
plt.ylabel("Average Cluster size")
plt.xlim(5400, 6000)
plt.xlabel("Voltage / V")
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))  # Create a new figure
for rpc in range(number):
    plt.plot(voltages[rpc][len(voltages[rpc])-len(eff[rpc]):], eff[rpc], label=labels[rpc], marker = "o", color=colour[rpc])
plt.title("Maximal Reconstruction Efficiency vs. Voltage")
plt.ylabel("Efficiency")
plt.xlim(5400, 6000)
plt.xlabel("Voltage / V")
plt.legend()
plt.show()



plt.figure(figsize=(10, 6))  # Create a new figure
for rpc in range(number):
    plt.plot(voltages[rpc][len(voltages[rpc])-len(stds[rpc]):], stds[rpc], label=labels[rpc], marker = "o", color=colour[rpc])
plt.title("Standard deviation of hit time vs. Voltage")
plt.ylabel("$\sigma$ / (25/32) ns")
plt.xlim(5000, 6000)
plt.xlabel("Voltage / V")
plt.legend()
plt.show()
